In [24]:
import numpy as np

# generate data

In [25]:
# In real-world scenarios, learning how the data was generated is impractical. Do not rely on this function while doing research.
def generate_data(dim, num):
    x = np.random.normal(0, 10, [num, dim])
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

# write your model class

In [26]:
from random import randint


# Modifying the overall structure is acceptable but not recommended
class SVM1:
    def __init__(self, dim, tol=1e-4, C=10):
        """
        Adding other parameters is acceptable but not necessary.
        """
        self.dim = dim
        self.w, self.b = np.zeros(dim), 0.0
        self.C, self.tol = C, tol

    def fit(self, X: np.ndarray, y: np.ndarray, eps=1e-4, max_round=5):
        """
        Fit the coefficients via your method1
        """
        N = X.shape[0]
        alpha = np.zeros((N, 1))
        b = 0.0
        K = X @ (X.transpose())
        E = np.zeros((N, 1))

        def gi(i):
            return K[i] @ (alpha * y) + b

        def calcE():
            nonlocal E
            for i in range(N):
                E[i] = gi(i) - y[i]

        calcE()

        def innerLoop(a1: int):
            nonlocal b

            def randa2():
                a2 = randint(0, N - 1)
                while a2 == a1:
                    a2 = randint(0, N - 1)
                return a2

            def run(a2: int):
                nonlocal b
                # calc new alpha1 and alpha2
                L, H = 0, 0
                if y[a1] != y[a2]:
                    L = max(0, alpha[a2] - alpha[a1])
                    H = min(self.C, self.C + alpha[a2] - alpha[a1])
                else:
                    L = max(0, alpha[a2] + alpha[a1] - self.C)
                    H = min(self.C, alpha[a2] + alpha[a1])
                if L >= H:
                    return 0

                def trunc(x):
                    return H if x > H else (L if x < L else x)

                K11, K22, K12 = K[a1][a1], K[a2][a2], K[a1][a2]
                eta = K11 + K22 - 2 * K12

                alpha2 = trunc(alpha[a2] + y[a2] * (E[a1] - E[a2]) / eta)
                alpha1 = alpha[a1] + y[a1] * y[a2] * (alpha[a2] - alpha2)

                # calc b
                b1new = (
                    -E[a1]
                    - y[a1] * K11 * (alpha1 - alpha[a1])
                    - y[a2] * K12 * (alpha2 - alpha[a2])
                    + b
                )
                b2new = (
                    -E[a2]
                    - y[a1] * K12 * (alpha1 - alpha[a1])
                    - y[a2] * K22 * (alpha2 - alpha[a2])
                    + b
                )
                if abs(alpha[a2] - alpha2) < eps:
                    return 0
                alpha[a1], alpha[a2] = alpha1, alpha2
                if 0 + eps < alpha[a1] < self.C - eps:
                    b = b1new
                elif 0 + eps < alpha[a2] < self.C - eps:
                    b = b2new
                else:
                    b = (b1new + b2new) / 2.0
                calcE()
                return 1

            a2 = a1
            if E[a1] >= 0:
                a2 = E.argmin()
            else:
                a2 = E.argmax()
            a2 = int(a2)
            if run(a2) == 0:
                a2 = randa2()
                return run(a2)
            else:
                return 1


        for round in range(max_round):
            tried, changed = 0, 0
            for i in range(N):
                if (y[i] * E[i] < -self.tol and alpha[i] < self.C - eps) or (
                    y[i] * E[i] > self.tol and alpha[i] > 0 + eps
                ):
                    tried += 1
                    changed += innerLoop(i)
            if tried == 0:
                break
            print(tried, changed, round)

        # generate model
        self.w = np.zeros(self.dim)
        for i in range(N):
            self.w += alpha[i] * y[i] * X[i]

        js = np.where(alpha > eps)[0]
        for j in js:
            self.b += y[j]
            for i in range(N):
                self.b -= alpha[i] * y[i] * K[i][j]
        self.b /= len(js)

    def predict(self, X):
        """
        Generate prediction probabilities on a new
        collection of data points by your model.
        """
        return np.sign(np.dot(X, self.w) + self.b)

In [27]:
class SVM2:
    def __init__(self, dim, C=10):
        """
        Adding other parameters is acceptable but not necessary.
        """
        self.dim = dim + 1
        self.w = np.zeros(self.dim)
        self.C = C

    def gradient(self, X: np.ndarray, y: np.ndarray):
        g = np.zeros(self.dim)
        cond = np.dot(X, self.w) * y
        for i in range(X.shape[0]):
            if cond[i] <= 1:
                g -= self.C * y[i] * X[i]
        g /= X.shape[0]
        g += self.w
        return g

    def loss(self, X: np.ndarray, y: np.ndarray):
        res = 0.0
        l = 1 - y * np.dot(X, self.w)
        res += self.C * np.sum(np.maximum(0, l))
        res /= X.shape[0]
        res += np.linalg.norm(self.w) / 2.0
        return res

    def fit(self, X: np.ndarray, y: np.ndarray, lr=0.005, round=2000):
        """
        Fit the coefficients via your method2
        """
        X = np.c_[np.ones(X.shape[0]), X]
        y = y.reshape(X.shape[0])
        ls = []
        for epoch in range(round):
            self.w -= lr * self.gradient(X, y)
            ls.append(self.loss(X, y))
        print(ls[-10:])

    def predict(self, X):
        """
        A same predict function with SVM1 is acceptable.
        """
        X = np.c_[np.ones(X.shape[0]), X]
        return np.sign(np.dot(X, self.w))

# construct and train your models

In [28]:
# generate data
X_data, y_data, mislabel = generate_data(20, 10000) 

# split data
# import sklearn.model_selection

def train_test_split(X, y, test_rate):
    Xy = np.concatenate((X, y), axis=1)
    np.random.shuffle(Xy)
    tot_size = Xy.shape[0]
    test_size = int(test_rate * tot_size)
    return Xy[test_size:, 0:-1], Xy[0:test_size, 0:-1], Xy[test_size:, -1:], Xy[0:test_size, -1:]

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_rate=0.2)

# X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_data, y_data, test_size=0.2)

print(mislabel)
X_train[:2], X_test[:2], y_train[:2], y_test[:2]

0.0379


(array([[ 11.15923943,   0.76187528,  -3.33158839, -19.40620148,
          -9.4610596 ,  -3.2952317 ,  26.56361226,  27.56207606,
          -7.32457128,   3.53412694,   4.06043977,   7.27333022,
          -0.56074602,  -3.2645011 ,  10.12536029,   5.96654826,
          -7.13506307, -17.67351485,   4.09080472, -12.71328366],
        [  7.32870877,   0.55597057,  11.84869995,  -8.05562468,
         -11.57699801,  -2.51182846,   3.45363605,   1.34505454,
          -0.17675613,  -3.68081452,  12.38722361, -14.94264211,
           8.60564711,   7.63959838,   6.67338868,  -2.3267233 ,
          -8.90444577,   4.99222461,  -1.50923607, -20.70451124]]),
 array([[ 12.90697769,   7.84737897,   6.74136957,   8.3815555 ,
           1.52959077,   4.06392425,   2.66887613,  13.21188961,
           2.43073194,  -5.94674224,  -3.55813606,  -3.97431792,
          -0.97208605, -15.50283716,   6.72866144,  -6.89203085,
          10.67696075, -12.55718006,   1.91555406, -23.68559407],
        [-15.7958614

In [29]:
# construct model and train (remember to record your time consumption)
model1 = SVM1(dim=X_test.shape[1]) 
model1.fit(X_train, y_train)

2355 457 0
2325 428 1
2648 642 2
2659 630 3
2932 766 4


In [30]:
model2 = SVM2(dim=X_test.shape[1]) 
model2.fit(X_train, y_train, round=500)

[1.9496443770684257, 1.949691015426595, 1.9496767376002324, 1.9495915297462918, 1.9495867556586999, 1.949644073649126, 1.9495846688559881, 1.9496117101041714, 1.9496252228036863, 1.9496300608372847]


In [31]:
from sklearn import svm
model_SVC = svm.SVC()
model_SVC.fit(X_train, y_train.flatten())

SVC()

In [32]:
from sklearn import svm
model_LSVC = svm.LinearSVC(dual=False)
model_LSVC.fit(X_train, y_train.flatten())

LinearSVC(dual=False)

# predict and compare your results

In [33]:
def check_model(model):
    # make prediction
    tpred = model.predict(X_train).reshape(y_train.shape)
    pred = model.predict(X_test).reshape(y_test.shape)

    # compare with generated label

    rate = np.count_nonzero(np.abs(pred - y_test) < 0.0005) / y_test.shape[0]
    trate = np.count_nonzero(np.abs(tpred - y_train) < 0.0005) / y_train.shape[0]

    print(pred[:10].flatten(), y_test[:10].flatten())

    print(rate, trate)

In [34]:
check_model(model1)
check_model(model2)
check_model(model_SVC)
check_model(model_LSVC)

[-1.  1.  1. -1. -1. -1. -1.  1. -1.  1.] [-1.  1.  1. -1. -1. -1. -1.  1.  1.  1.]
0.899 0.8925
[-1.  1.  1. -1. -1. -1. -1.  1. -1.  1.] [-1.  1.  1. -1. -1. -1. -1.  1.  1.  1.]
0.9555 0.956125
[-1.  1.  1. -1. -1. -1. -1.  1. -1.  1.] [-1.  1.  1. -1. -1. -1. -1.  1.  1.  1.]
0.9405 0.966625
[-1.  1.  1. -1. -1. -1. -1.  1. -1.  1.] [-1.  1.  1. -1. -1. -1. -1.  1.  1.  1.]
0.9565 0.95375
